In [1]:
import os
from dotenv import load_dotenv
import gradio as gr
import requests
import whisper

In [2]:
#Load environment variables from .env
load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")

In [3]:
#Load Whisper model
whisper_model = whisper.load_model("base")

In [4]:
def voice_to_deepseek(audio_file):
    if not audio_file:
        return "Please provide a voice input."

    # Step 1: Transcribe audio to text using Whisper
    result = whisper_model.transcribe(audio_file)
    transcribed_text = result["text"]

    if not transcribed_text.strip():
        return "Couldn't understand the audio."

    # Step 2: Send transcribed text to DeepSeek via OpenRouter
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "deepseek/deepseek-r1-0528:free",
        "messages": [
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": transcribed_text}
        ]
    }

    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers=headers,
        json=payload
    )

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error {response.status_code}: {response.text}"

In [5]:
#Gradio Voice-Only UI
iface = gr.Interface(
    fn=voice_to_deepseek,
    inputs=gr.Audio(type="filepath", label="Speak your question"),
    outputs=gr.Textbox(label="DeepSeek Response"),
    title="Voice Q&A with DeepSeek + Whisper",
    description="Speak your question. Whisper transcribes it, DeepSeek answers it via OpenRouter."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


C:\AI assiss\class_env\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
